In [1]:
import scipy.io
import numpy as np

# Carregar dades des del fitxer .mat 
mat = scipy.io.loadmat(r"C:\Users\joana\Documents\Projectes personals\TR\matlab\matlab\emnist-letters.mat")

# Accedir a les dades
X_letters_train = mat['dataset']['train'][0,0]['images'][0,0]
y_letters_train = mat['dataset']['train'][0,0]['labels'][0,0]
X_letters_test = mat['dataset']['test'][0,0]['images'][0,0]
y_letters_test = mat['dataset']['test'][0,0]['labels'][0,0]

# Reformatar
X_letters_train = X_letters_train.reshape((-1, 28, 28), order='F').astype(np.uint8)
X_letters_test = X_letters_test.reshape((-1, 28, 28), order='F').astype(np.uint8)
y_letters_train = y_letters_train.flatten() - 1  # Canviar de 1-26 a 0-25
y_letters_test = y_letters_test.flatten() - 1

In [2]:
mat_digits = scipy.io.loadmat(r"C:\Users\joana\Documents\Projectes personals\TR\matlab\matlab\emnist-digits.mat")

# Dades d'entrenament
X_digits_train = mat_digits['dataset']['train'][0,0]['images'][0,0]
y_digits_train = mat_digits['dataset']['train'][0,0]['labels'][0,0]

# Dades de test
X_digits_test = mat_digits['dataset']['test'][0,0]['images'][0,0]
y_digits_test = mat_digits['dataset']['test'][0,0]['labels'][0,0]

In [3]:
# Format de les imatges (vector --> matriu 28x28)
X_digits_train = X_digits_train.reshape(-1, 28, 28, order='F').astype(np.uint8)
X_digits_test = X_digits_test.reshape(-1, 28, 28, order='F').astype(np.uint8)

# Ajusta etiquetes: els dígits van del 0 al 9, no cal restar 1
y_digits_train = y_digits_train.flatten()
y_digits_test = y_digits_test.flatten()

# Reassignar etiquetes dels dígits perquè no se solapin amb les lletres
y_digits_train += 26
y_digits_test += 26

In [4]:
from sklearn.preprocessing import OneHotEncoder

# Inicialitzem l'encodificador
encoder = OneHotEncoder(sparse_output=False, categories='auto')

# Ajustem i transformem les etiquetes (cal reshaping per a columnes)
y_train_cat_digits = encoder.fit_transform(y_digits_train.reshape(-1, 1))
y_test_cat_digits = encoder.transform(y_digits_test.reshape(-1, 1))

print("Etiqueta original:", y_digits_train[2])
print("One-hot encoded:", y_train_cat_digits[2])

Etiqueta original: 32
One-hot encoded: [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout # type: ignore

# Defineixo la CNN
model_cnn = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2,2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),  # Redueix overfitting
    Dense(y_train_cat_digits.shape[1], activation='softmax')  # Sortida: tantes neurones com classes
])

# Compilo
model_cnn.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Entreno
history_cnn = model_cnn.fit(
    X_digits_train, y_train_cat_digits,
    epochs=10,
    batch_size=128,
    validation_data=(X_digits_test, y_test_cat_digits),
    verbose=1
)

# Avaluació
test_loss, test_acc = model_cnn.evaluate(X_digits_test, y_test_cat_digits, verbose=0)
print(f"Precisió CNN: {test_acc:.4f}")

c:\Users\joana\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 28s 14ms/step - accuracy: 0.9409 - loss: 0.3303 - val_accuracy: 0.9876 - val_loss: 0.0489
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 35s 19ms/step - accuracy: 0.9804 - loss: 0.0740 - val_accuracy: 0.9912 - val_loss: 0.0310
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 43s 23ms/step - accuracy: 0.9848 - loss: 0.0562 - val_accuracy: 0.9920 - val_loss: 0.0286
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 41s 22ms/step - accuracy: 0.9871 - loss: 0.0479 - val_accuracy: 0.9922 - val_loss: 0.0299
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 40s 21ms/step - accuracy: 0.9886 - loss: 0.0418 - val_accuracy: 0.9936 - val_loss: 0.0245
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 42s 22ms/step - accuracy: 0.9897 - loss: 0.0386 - val_accuracy: 0.9931 - val_loss: 0.0271
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 63s 33ms/step - accuracy: 0.9903 - loss: 0.0361 - val_accuracy: 0.9929 - val_loss: 0.0297
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 54s 29ms/step - accuracy: 0.9913 -